## Setup

In [ ]:
from functions import Model, get99, splitData, dropMissingValues, z_score, outliers_knn, outliers_dbscan, getNoise, getRelFeatures, drop_features, getCombinations, get_unique_list, bold, blue, red, green, getBestModel, np_to_df, df_to_np, drop99_all, z_score_individual, train_test_to_df
import pandas as pd
from sklearn.linear_model import LinearRegression
import warnings
import json
import numpy as np
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from functions import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

In [ ]:
# JSON file
f = open ('data/json/input.json', "r")
  # Reading from file
PARAMETERS = json.loads(f.read())

SPLIT_SIZE = PARAMETERS["SPLIT_SIZE"]

OUTLIER99 = PARAMETERS["OUTLIER99"] 
OUTLIER_ZSCORE = {"STD" : PARAMETERS["OUTLIER_ZSCORE"]["STD"]}
OUTLIER_ZSCORE_IND = PARAMETERS["OUTLIER_ZSCORE_IND"]
OUTLIER_KNN = {"K": PARAMETERS["OUTLIER_KNN"]["K"], "X": PARAMETERS["OUTLIER_KNN"]["X"]}
OUTLIER_DBSCAN = {"K": PARAMETERS["OUTLIER_DBSCAN"]["K"], "X": PARAMETERS["OUTLIER_DBSCAN"]["X"], "EPS": PARAMETERS["OUTLIER_DBSCAN"]["EPS"], "MIN_SAMPLES":  PARAMETERS["OUTLIER_DBSCAN"]["MIN_SAMPLES"]}
OUTLIER_NOISE = {"CV": PARAMETERS["OUTLIER_NOISE"]["CV"]}

FEATURE_SEL_CORR = {"THRESHOLD_CORR": PARAMETERS["FEATURE_SEL_CORR"]["THRESHOLD"]}

f.close()

#TODO FEATURE
#TODO Regressionmodels
#TODO TESTDATA --> path to csv-file


In [ ]:
#Get raw Data
df = pd.read_csv("data/csv/house_data_training.csv", sep=';') 
# remove unnamed column
df = df.iloc[:, 1:]
#Transform string to datetime
df["date"] = pd.to_datetime(df["date"])

maeList = []#for model selection

In [ ]:
df = dropMissingValues(df) 

## Outlier Detection

In [ ]:
# Outlier Detection
outlier_dict_all = {}

if OUTLIER99 == True:
    print("outlier_99")
    outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
    df = drop99_all(df, outlier_list_99 )
    #outlier_dict_all["99_mean"] = outlier_list_99
    #outlier_dict_all["99_drop"] = outlier_list_99
    print(green('done'), "\n")

print('starting z_score')
for p in OUTLIER_ZSCORE["STD"]:
    print(p)
    outlier_list_z_score = z_score(df, p)
    outlier_dict_all["z_score_"+str(p)] = outlier_list_z_score

if OUTLIER_ZSCORE_IND == True:
    outlier_list_z_score_ind = z_score_individual(df)

print(green('done'), '\n\nstarting knn')
for k in OUTLIER_KNN["K"]:
    for o in OUTLIER_KNN["X"]:
        print(k, o)
        outlier_list_knn = outliers_knn(df, k, o, SPLIT_SIZE)
        outlier_dict_all["knn_"+str(k)+"_"+str(o)] = outlier_list_knn

print(green('done'), '\n\nstarting db_scan')
for k in OUTLIER_DBSCAN["K"]:
    for o in OUTLIER_DBSCAN["X"]:
        for e in OUTLIER_DBSCAN["EPS"]:
            for m in OUTLIER_DBSCAN["MIN_SAMPLES"]:
                print(k, o, e, m)
                outlier_list_dbscan = outliers_dbscan(df, k, o, e, m, SPLIT_SIZE)
                outlier_dict_all["db_scan"] = outlier_list_dbscan

print(green('done'), '\n\nstarting noise')
for cv in OUTLIER_NOISE["CV"]:
        print(cv)
        outlier_list_noise = getNoise(df, cv)
        outlier_dict_all["noise_"+str(cv)] = outlier_list_noise
print(green('done'))

In [ ]:
outlier_lists_all = list(outlier_dict_all.values())
outlier_lists_keys = list(outlier_dict_all.keys())
combination_list = []
for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
    if not i == 1: #emtpy tuple
        combination_list.append(combo)
        # print('combo #{}: {}'.format(i, combo))
print("...")
for c in combination_list[10:]:
    print(bold("Combination: "), blue(c),"\n",)
print(bold(len(combination_list)), " total Combinations")

In [ ]:
model_obj_list = []
for i, c in enumerate(combination_list):
    print(f'\n{bold(i)}: {blue(c)}')
    outlier_list = get_unique_list(outlier_dict_all, c)
    try: 
        
        # if "99_mean" in c:
        #     X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
        # else:
        #     X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")
        #df_price_rem = df.drop(columns=["price"])
        # print(df_price_rem[:2])
        #print(df[:1])
        X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop") #TODO drop entfernen
        #print("test")
        #df_linear = train_test_to_df(X_train, X_test, y_train, y_test, df.columns.to_list())
        #print(df_linear[:1])
        
        #TODO Regressionmodels
        model = LinearRegression()
        obj = Model(model, (X_train, X_test, y_train, y_test), c, df, df.columns.to_list())
        model_obj_list.append(obj)

        # ''''''     
        # df_lasso = train_test_to_df(X_train, X_test, y_train, y_test, df.columns.to_list())
        # #print(df_lasso[:1])
        # # X = np.append(X_train, X_test, axis=0)
        # # y = np.append(y_train, y_test.reshape(len(y_test), 1), axis=0)
        # # X_y = np.append(X, y, axis=1)
        # # features = df.columns.to_list()
        # # df_adsa = pd.DataFrame(X_y, columns=features)

        # X_l, y_l = df_lasso.drop(columns=["price", "date", "id"]), df_lasso["price"]
        # scaler = StandardScaler().fit(X_l)
        # X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X_l, y_l, test_size = SPLIT_SIZE, random_state=1) #TODO 
        # #rint(X_train_l)
        # X_train_l = scaler.transform(X_train_l)
        # X_test_l = scaler.transform(X_test_l)
        # reg = Lasso(alpha=1)
        # obj_l = Model(reg, (X_train_l, X_test_l, y_train_l, y_test_l), c, df_lasso, X_l.columns.to_list())
        # model_obj_list.append(obj_l)
        # ''''''
    except Exception as e:
        print(bold("Error"), red(e))

In [ ]:
#model_obj_list = []
for i, c in enumerate(combination_list):
    print(f'\n{bold(i)}: {blue(c)}')
    outlier_list = get_unique_list(outlier_dict_all, c)
    try: 
        X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")
        df_lasso = train_test_to_df(X_train, X_test, y_train, y_test, df.columns.to_list())
        #print(df_lasso[:1])
        # X = np.append(X_train, X_test, axis=0)
        # y = np.append(y_train, y_test.reshape(len(y_test), 1), axis=0)
        # X_y = np.append(X, y, axis=1)
        # features = df.columns.to_list()
        # df_adsa = pd.DataFrame(X_y, columns=features)

        X_l, y_l = df_lasso.drop(columns=["price", "date", "id"]), df_lasso["price"]
        scaler = StandardScaler().fit(X_l)
        X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X_l, y_l, test_size = SPLIT_SIZE, random_state=1) #TODO 
        #rint(X_train_l)
        X_train_l = scaler.transform(X_train_l)
        X_test_l = scaler.transform(X_test_l)
        reg = Lasso(alpha=1)
        obj_l = Model(reg, (X_train_l, X_test_l, y_train_l, y_test_l), c, df_lasso, X_l.columns.to_list())
        model_obj_list.append(obj_l)
        ''''''
    except Exception as e:
        print(bold("Error"), red(e))

In [ ]:
# def getDF(self):
#     X_train, X_test, y_train, y_test = self.get_data()
#     print(len(y_test))
#     #print(y_train.shape, y_test.reshape(len(y_test), 1).shape)
#     X = np.append(X_train, X_test, axis=0)
#     y = np.append(y_train, y_test.reshape(len(y_test), 1), axis=0)
#     X_y = np.append(X, y, axis=1)
#     features = df.columns.to_list()
#     features.remove("price")
#     df = pd.DataFrame(X_y, columns=features)
#     return df

In [20]:
#Summary
df_summary = pd.DataFrame(columns=['model', 'combo', 'mae', 'score'])
for o in model_obj_list:
    #print(f'{o.get_type()}, {o.get_comb()}')
    o.train()
    mae, score = o.summary()
    comb = o.get_comb()
    row = {'model': o.get_model(), 'combo': comb, 'mae': mae, 'score': score}
    df_summary = df_summary.append(row, ignore_index=True)

df_summary.sort_values("mae")

,model,combo,mae,score
29,Lasso(alpha=1),"(z_score_10.1, knn_3_200, db_scan, noise_5)",0.67,-0.001289
27,Lasso(alpha=1),"(z_score_10.1, db_scan, noise_5)",0.67,-0.001289
25,Lasso(alpha=1),"(z_score_10.1, knn_3_200, db_scan)",0.67,-0.000193
24,Lasso(alpha=1),"(db_scan, noise_5)",0.67,-0.001379
20,Lasso(alpha=1),"(z_score_10.1, db_scan)",0.67,-0.000193
28,Lasso(alpha=1),"(knn_3_200, db_scan, noise_5)",0.67,-0.001379
22,Lasso(alpha=1),"(knn_3_200, db_scan)",0.68,-0.000687
17,Lasso(alpha=1),"(db_scan,)",0.68,-0.000687
26,Lasso(alpha=1),"(z_score_10.1, knn_3_200, noise_5)",0.72,-0.000212
23,Lasso(alpha=1),"(knn_3_200, noise_5)",0.72,-0.00009


In [23]:
print(model_obj_list[1])
lassotest = model_obj_list[1]

X_train, X_test, y_train, y_test = lassotest.get_data()
lassotest.get_model().fit(X_train, y_train)
#print(reg.score(X_test, y_test))
y_pred = reg.predict(X_test)
print("mae", mean_absolute_error(y_test, y_pred))

Model: LinearRegression() Combination: ('knn_3_200',) Features: 24


ValueError: X has 24 features, but Lasso is expecting 22 features as input.

In [ ]:
#row
mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
#print(mae_best)
#score_best = df_summary[df_summary.score==df_summary.score.max()]
best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
print(best_model_obj)
#rint(f'{bold("Lowest mae:")} {mae_best}')
#model_obj_list

## Feature Selection

In [ ]:
model_obj_list_best = [best_model_obj]
X_train, X_test, y_train, y_test = best_model_obj.get_data()

df_feature_reduction = train_test_to_df(X_train, X_test, y_train, y_test, best_model_obj.get_features())
print(df_feature_reduction.columns)
#print(df_feature_reduction)

for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
    #print(threshold, type(threshold))
    try: 
        rel_features = getRelFeatures(df_feature_reduction, threshold)
        print(len(rel_features), rel_features)
        df_feature_reduction = df_feature_reduction[rel_features]
        X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
        best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction)
        model_obj_list_best.append(best_model_obj_new)

    except Exception as e:
        print(bold("Error"), red(e))
print(model_obj_list_best)

In [ ]:
#Summary
df_summary_best = pd.DataFrame(columns=['model', 'combo', 'mae', 'score', 'features'])
for o in model_obj_list_best:
    #print(f'{o.get_type()}, {o.get_comb()}')
    o.train()
    mae, score = o.summary()
    comb = o.get_comb()
    features = o.get_df().columns.to_list()
    if "price" in features:
        features.remove("price")
    row = {'model': o.get_model(),'combo': comb, 'mae': mae, 'score': score, 'features': features}
    df_summary_best = df_summary_best.append(row, ignore_index=True)

df_summary_best.sort_values("mae")

In [ ]:
#row
mae_best = df_summary_best[df_summary_best.mae==df_summary_best.mae.min()]
#score_best = df_summary_best[df_summary_best.score==df_summary_best.score.max()]
best_model_obj = getBestModel(model_obj_list_best, df_summary_best, mae_best.index[0])
print(best_model_obj)
print(f'{bold("Lowest mae:")} {mae_best}')


In [ ]:
df.columns

In [21]:
X_train, X_test, y_train, y_test =  best_model_obj.get_data()
X = np.append(X_train, X_test, axis=0)
y = np.append(y_train, y_test.reshape(len(y_test), 1), axis=0)
X_y = np.append(X, y, axis=1)
features = df.columns.to_list()
df_adsa = pd.DataFrame(X_y, columns=features)

df_adsa

NameError: name 'best_model_obj' is not defined

In [ ]:
rel = ['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'dis_super', 'view', 'condition',
       'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
       'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'ahf1', 'ahf2',
       'ahf3']
from functions import train_test_split
from sklearn.preprocessing import StandardScaler
X, y = df_adsa.drop(columns=["price", "date", "id"]), df_adsa["price"]
scaler = StandardScaler().fit(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = SPLIT_SIZE, random_state=1) 

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

reg = Lasso(alpha=1)
reg.fit(X_train, y_train)
print(reg.score(X_test, y_test))
y_pred = reg.predict(X_test)
print("dsds", mean_absolute_error(y_test, y_pred))

## Regressionmodels

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
poly.fit_transform(best_model_obj_new.get_df())




## Evaluatuion

## Testdata

In [ ]:
# combination_list = []
# df_feature_reduction = best_model_obj.get_df()

# for i, combo in enumerate(getCombinations(df_feature_reduction.columns.to_list()), 1):
#     if not i == 1: #emtpy tuple
#         combination_list.append(combo)
#         # print('combo #{}: {}'.format(i, combo))
# print("...")
# for c in combination_list[10:]:
#     print(bold("Combination: "), blue(c),"\n",)
# print(bold(len(combination_list)), " total Combinations")

In [ ]:
# model_obj_list_best = []
# df_feature_reduction = best_model_obj.get_df()
# for i, c in enumerate(combination_list):
#     print(f'\n{bold(i)}: {blue(c)}')
    
#     try: 
#         X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
#         best_model_obj_copy = best_model_obj.copy()
#         best_model_obj_copy.set_data(X_train, X_test, y_train, y_test)

#         model_obj_list_best.append(obj)

#     except Exception as e:
#         print(bold("Error"), red(e))

In [ ]:
# df_feature_reduction = best_model_obj.get_df()
# getCombinations(df_feature_reduction.columns.to_list())

# for feature in list(df_feature_reduction.columns.to_list()):
#     df_feature_reduction.drop(columns=feature)
#     X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
#     best_model_obj_copy = best_model_obj.copy()
#     best_model_obj_copy.set_data(X_train, X_test, y_train, y_test)

In [ ]:
# #Wrapper approaches
# #Backward elimination using Recursive feature elimination¶
# from sklearn.svm import SVC
# from sklearn import feature_selection
# import pandas as pd
# import numpy as np


# x, y = df.drop(columns=["price"]), df["price"]
# # x_y= np.concatenate((x.reshape(150,4),y.reshape(150,1)),1)
# x_y= np.concatenate((x,y),1)
# #print(x_y)
# #print(iris.target_names)

# #Create column list
# df_columns=df.columns
# # df_columns.append("Label")
# # #Create PANDAS data frame
# # df = pd.DataFrame(x_y,columns=df_columns)
# #Map label index to label name
# df['price']=df['price'].map(lambda x: iris.target_names[int(x)])
# print("\nOriginal Iris Data Set:")
# print(df)

# #Create the RFE object and rank features
# num_features=2
# svc = SVC(kernel="linear", C=1)
# rfe = feature_selection.RFE(estimator=svc, n_features_to_select=num_features, step=1)
# rfe.fit(x, y)
# #print("Selected features will have a ranking=1 and support=TRUE")
# #print(iris.feature_names," ",rfe.ranking_," ",rfe.support_)
# #print(x.shape)

# #extend column-mask by one column for the label (always true)
# column_mask=np.append(rfe.support_,True)
# #use list column_mask to mask df-columns list
# reduced_iris_features = [df_columns[i] for i in range(len(df_columns)) if column_mask[i]]
# print("Reduced_iris_features: ",reduced_iris_features)
# # use reduced_iris_features to reduce the data frame
# reduced_df=df[reduced_iris_features]

# print("\nIris Data Set reduced to ",num_features," features: \n",reduced_df)

In [ ]:
#TODO sqft_living wenn price auffällig niedrig obwohl m² hoch clsuter
#TODO PCA macht warscheinlich wenig sinn